Start with one point, apply convolution with some kernel to find high value on the pixel

Check PCA documentation, get components then coefficients

[MN, T] -> [MN, components] X [components, T]

When we get fit transform could track pixel or time trace, keep track pixel and reduce time points
One option is PCA on data or data transpose, check dimension of results include number of pixels rather than the number of frames
[500, 491, 491]
data[0] = [491, 491]

PCA should be [491 * 491] if it is [500] it was time

#Problem 5: Matrix factorization
The above procedure is common in many labs, however can cause errors due to the dependency on the summary image and the multiple stages of processing. Matrix factorization has emerged as an alternative approach for identifying ROIs from the full spatio-temporal video. Here we will explore three different types of factorization and compare the results using a table.

## Part A:
Run PCA on the pixels-by-time matrix obtained by vectorizing each frame into a column vector. How does the result change with the number of principal components you choose?

In [4]:
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import tifffile
from google.colab import drive
from PIL import Image
import numpy as np
import sklearn
from sklearn import cluster, decomposition

# Find the tif file in google drive
drive.mount('/content/drive')
file = "/content/drive/MyDrive/Neural_Signals_and_Computation_HW1/TEST_MOVIE_00001-small.tif"

# Load tif file into numpy array and image
data = tifffile.imread(file)

# Vectorize each frame into a column vector thenc combine each column together
# Matrix -> (500*500, 500) or (MN, T)
full_data = []
for k in range(len(data)):
  column = np.array(data[k]).flatten()
  full_data.append(column)
full_data = np.array(full_data).transpose()

# Create a square layout for the plot
layout = go.Layout(
    width=data.shape[1],  # Set the width of the plot
    height=data.shape[2],  # Set the height of the plot
    xaxis=dict(range=[0, data.shape[1]]),  # Set the x-axis range to match the width of the image
    yaxis=dict(range=[0, data.shape[2]]),  # Set the y-axis range to match the height of the image
    margin=dict(l=0, r=0, t=0, b=0),  # Set the margins to 0 to remove unnecessary spacing
)

# Check when we choose number of components k = 2 to 7
for i in range(5):
  n_components = i + 2
  pca_estimator = decomposition.PCA(
    n_components=n_components, svd_solver="randomized", whiten=True
  )
  pca_result_mn = pca_estimator.fit_transform(full_data)
  pca_result_f = pca_estimator.components_

  result = np.matmul(pca_result_mn, pca_result_f)

  pca_data = []
  for j in range(result.shape[1]):
    pca_data.append(result[:, j].reshape((500, 500)))
  pca_data = np.array(pca_data)

  # Plot the summary images
  print("Using " + str(i + 2) + " components: ")
  fig_mean = go.Figure(data=go.Heatmap(z=pca_data[0]), layout=layout)
  fig_mean.show()

# Original Data Image Frame 0
print("Original Data")
fig_mean = go.Figure(data=go.Heatmap(z=data[0]), layout=layout)
fig_mean.show()

Output hidden; open in https://colab.research.google.com to view.

Using PCA involves reducing the dimension of the image per frame to however many components we wish to analyze that covers as much variance of the data. As we increase the number of components, we reduce the degree of dimension reduction we do, thus somewhat turning the image more into what we originally started with. With very low number of components, it is unable to capture all the variance and detail in the original as much as higher components can.